In [29]:
# Import libraries
import theano
import theano.tensor as T
import sklearn.metrics
import numpy as np

In [30]:
# Regularization
def L2(x):
    return T.sum(x**2)

def L1(x):
    return T.sum(abs(x))

In [31]:
# Examples, features and hidden neurons
examples = 1000
features = 100
hidden = 2

In [32]:
# Create dataset
D = (
np.random.randn(examples,features),
    np.random.randint(size = examples,
                     low = 0,
                     high = 2
                     )
)

In [33]:
# Training steps
training_steps = 10000
# Learning rate
learning_rate = 0.1
# Regularization
lambda_reg = 0.01

In [34]:
# Create feature matrix
x = T.dmatrix('x')
# Create label vector
y = T.dvector('y')

# Create weight matrix layer 1
w1 = theano.shared(
np.random.randn(features, hidden),
    name = 'w1'
)
# Create bias for layer 1
b1 = theano.shared(
np.random.randn(hidden), name = 'b1'
)

# Create weight matrix layer 2
w2 = theano.shared(
np.random.randn(hidden), name = 'w2'
)
# Create bias for layer 2
b2 = theano.shared(0., name = 'b2')

# Create activation functions
p1 = T.tanh(T.dot(x,w1)+b1)
p2 = T.tanh(T.dot(p1, w2)+b2)

# Prediction
prediction = p2 > 0.5

# Error
error = T.nnet.binary_crossentropy(p2,y)

# Loss
loss = error.mean() + lambda_reg * (
(L2(w2) + L2(w1) + L1(w1) + L1(w2))
)

# Gradients
gw1,gb1,gw2,gb2 = T.grad(loss, [w1,b1,w2,b2])

In [35]:
# Training function
train = theano.function(
inputs = [x,y], outputs = [p2,error],
    updates = ((w1,w1 - learning_rate*gw1),
              (b1, b1 - learning_rate*gb1),
              (w2, w2 - learning_rate*gw2),
              (b2, b2 - learning_rate*gb2)
              )
)

# Prediction function
predict = theano.function(
inputs = [x], outputs = prediction
)

In [36]:
# Display pre training accuracy
print("Accuracy before training = {}".format(
sklearn.metrics.accuracy_score(D[1],
                              np.array(predict(D[0])).ravel())
))

Accuracy before training = 0.487


In [39]:
# Train network
for i in range(training_steps):
    prediction, error = train(D[0],D[1])

In [40]:
# Display post training accuracy
print("Accuracy after training = {}".format(
sklearn.metrics.accuracy_score(D[1],
                              np.array(predict(D[0])).ravel())
))

Accuracy after training = 0.524
